# Top1%: FastAI Beginner Tutorial

This is my first notebook submission. Please let me know if i can improve incase i have left some things unexplained.
We are not USING EDA here. Only automatic preprocessing data in fastai and training model and getting score of top1%.


Here, we are using dataset which is a bit extension of classic titanic dataset. you can check out this: https://www.kaggle.com/pavlofesenko/titanic-extended

And It is fair to use this, i reckon. We are not altering test data at all. Only using new features(Like how we generate new features). Number of rows remains same in both test and training data.

We will be using newly released fastai2.0 library and standard sklearn. fastai2.0 is worth checking out https://docs.fast.ai/ It is more fast and efficient. With this, we can minimize our effort for preprocessing and gain better results.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Installing fastai2, yes its not preinstalled. We would need to install on our own.

Warning: If you get some sort of error in importing fastai modules. I would recommend changing notebook environment to

    Always use latest environment Always get the latest package versions, but you may have to modify your code.



In [ ]:
!pip install fastai2

In [ ]:
!pip install fastcore==0.1.35 ##Currently supported with fastai2

In [ ]:
import fastcore
fastcore.__version__

In [ ]:
import fastai2
fastai2.__version__

#  Importing and Loading files

In [ ]:
from fastai2.tabular.all import *

In [ ]:
df_test= pd.read_csv('/kaggle/input/titanic-extended/test.csv')
df_train= pd.read_csv('../input/titanic-extended/train.csv')
df_train.head()

Just taking abit statistical look into data.

In [ ]:
df_train.describe()

Calculating the average null values we have in our data. It is important to know. Just to get intuition about data 

In [ ]:
df_train.isnull().sum().sort_index()/len(df_train)

Getting names of columns according to its datatype.

This will allow us to understand which feature is int, float, and object(categorical)


In [ ]:
df_train.dtypes
g_train =df_train.columns.to_series().groupby(df_train.dtypes).groups
g_train

Now this might be a bit puzzling if you are new to fastai. And its completely alright.

cat_names refers to the features which are categorical.

cont_names refers to the features which are continuous. For example : int and float

fastai needs them in order to do preprocessing for you properly


In [ ]:
cat_names= [
        'Name', 'Sex', 'Ticket', 'Cabin', 
        'Embarked', 'Name_wiki', 'Hometown', 
        'Boarded', 'Destination', 'Lifeboat', 
        'Body'
]

cont_names = [ 
    'PassengerId', 'Pclass', 'SibSp', 'Parch', 
    'Age', 'Fare', 'WikiId', 'Age_wiki','Class'
 ]



# FastAI2

now we are diving into fastai2. looking down at code. Might be new and intimidating. But no worries,
following the documentation helped me. https://docs.fast.ai/tutorial.tabular

Here we are using fastai TabularPandas library. Which will do all the preprocessing for us.
before that splitting our data into validation set to have a fair amount of idea that we are not overfitting the data. valid_pct= 0.2 means (as you may have guessed by now) it means 20% validation data.

y_names, using column we are targetting to be predicted.
procs are [Categorify, FillMissing,Normalize], Convering objects into category, filling the missing value and normalizing the content for quick processing.


NOTE:
Tabularpandas function creates a new column if there is any missing value spotted in any particular feature(only float or int column). for example: lets say Fare has a missing value somewhere. It will create a new column named 'Fare_na' where it will have integer values (1 or 2). 

In [ ]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

to = TabularPandas(df_train, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Survived',
                   splits=splits)

Here you can see 'Age_na', 'WikiId_na', 'Age_wiki_na', 'Class_na' . Which were created becoz their columns had missing values

In [ ]:
#df_train.dtypes
g_train =to.train.xs.columns.to_series().groupby(to.train.xs.dtypes).groups
g_train

In [ ]:
to.train.xs.Age_na.head()

Generally what happens is, we cannot use the output of tabularpandas straightaway. So we need to access it as follows.. just taking peek at train data(for validation data replace train with valid)

In [ ]:
to.train

using 'xs' to get the table in the manner we are used to see it. Notice that it is without 'Survived' column

In [ ]:
to.train.xs

using the same for target value

In [ ]:
to.train.ys.values.ravel()

Using RandomForestClassifier for solving this problem

and taking out X_train and y_train values like we showed before


In [ ]:
### RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_valid, y_valid = to.valid.xs, to.valid.ys.values.ravel()

We have table without any hardcore preprocessing all we did was just to use fastai tabular function to get this

In [ ]:
X_train.head()

In [ ]:
rnf_classifier= RandomForestClassifier(n_estimators=100, n_jobs=-1)
rnf_classifier.fit(X_train,y_train)

We just Trained randomforest classifier and predicting accuracy on validation set

In [ ]:
y_pred=rnf_classifier.predict(X_valid)


from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_valid)

WOW, we get the accuracy of 0.98 on validation dataset this means on leader board getting rank of 200 something our of 19,000. 

# TEST dataset

In [ ]:
df_test.head()

doing the same preprocessing we did for training set

In [ ]:
df_test.dtypes
g_train =df_test.columns.to_series().groupby(df_test.dtypes).groups
g_train

In [ ]:
cat_names= [
        'Name', 'Sex', 'Ticket', 'Cabin', 
        'Embarked', 'Name_wiki', 'Hometown', 
        'Boarded', 'Destination', 'Lifeboat', 
        'Body'
]

cont_names = [ 
    'PassengerId', 'Pclass', 'SibSp', 'Parch', 
    'Age', 'Fare', 'WikiId', 'Age_wiki','Class'
 ]

avoiding, y_name and split for obvious reasons

In [ ]:
test = TabularPandas(df_test, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   )

getting the table value for test dataset. we used train.xs becoz it lets us extact the data we had given the tabularpandas function


In [ ]:
X_test= test.train.xs

In [ ]:
X_test.head()

In [ ]:
X_test.dtypes
g_train =X_test.columns.to_series().groupby(X_test.dtypes).groups
g_train

Now you would be thinking why we are dropping the 'Fare_na' column. The reason is, originally we trained model on 24 features. But as you know that tabularpandas function creates a new column if there is any missing value spotted in any particular feature.

Here, our test data had some Fare values missing so the function created the boolean column this lead to 25 features. If you try to pass without dropping Fare_na column, it will give you the error that model was trained on 24 features and we are passing 25. That's why dropping Fare_na feature. And dont worry it doesnt have any effect on the output.


In [ ]:
X_test= X_test.drop('Fare_na', axis=1)

In [ ]:
y_pred=rnf_classifier.predict(X_test)


In [ ]:
y_pred= y_pred.astype(int)

here is our output

In [ ]:
output= pd.DataFrame({'PassengerId':df_test.PassengerId, 'Survived': y_pred})
output.to_csv('my_submission_titanic.csv', index=False)
output.head()